<a href="https://colab.research.google.com/github/shantanu2383/Data-Science-Tools/blob/main/Factiva_PDF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/gdrive')



In [ ]:
!pip install PyPDF2
import PyPDF2
import re
import pandas as pd
import glob
import os

# Define the regex patterns
word_count_pattern = r'(\d{1,3}(?:,\d{3})*|\d+) words'
date_pattern = r'\d{1,2} [A-Z][a-z]+ \d{4}(?!\d{2}:\d{2})'
publisher_pattern = r'(?<=\n)' + date_pattern + r'\n(?:\d{2}:\d{2}\n)?(\w+(?:\s+\w+){0,3})'

# Initialize an empty list to store the rows of data
data = []

for file_name in os.listdir(pdf_folder):
    if file_name.endswith(".pdf"):
        # Open the PDF file
        with open(os.path.join(pdf_folder, file_name), 'rb') as pdf_file:

          # Create a PDF reader object
          pdf_reader = PyPDF2.PdfReader(pdf_file)

          # Get the number of pages in the PDF file
          num_pages = len(pdf_reader.pages)

          # Loop through all pages and extract text
          page_text = []
          for page_num in range(num_pages):
              page_obj = pdf_reader.pages[page_num]
              text = page_obj.extract_text()
              page_text.append(text)

          # Close the PDF file
          pdf_file.close()

          # Join all text from all pages into one long string
          full_text = ' '.join(page_text)

          # Split the text into articles
          separator_pattern = r'Document [a-zA-Z0-9]{5,}'
          articles_text = re.split(separator_pattern, full_text)[1:]

          # Iterate through each article and extract the desired information
          for article in articles_text:
              # Extract the word count
              word_count_match = re.search(word_count_pattern, article)
              word_count = word_count_match.group(1) if word_count_match else 'not found'

              # Extract the date
              date_match = re.search(date_pattern, article)
              date = date_match.group() if date_match else 'not found'

              # Extract the publisher
              publisher_match = re.search(publisher_pattern, article, flags=re.MULTILINE)
              publisher = publisher_match.group(1) if publisher_match else 'not found'

              # Append the data to the list
              data.append([article, word_count, date, publisher])

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=['article', 'word_count', 'date', 'publisher'])


In [ ]:
df.to_csv(filepath+ 'FACTIVA_ARTICLES.csv', index=False)
